In [ ]:
import tensorflow as tf
import numpy as np
import os
import random

from evaluator import final_score
from DF_TAR import DF_TAR
from data_loader import data_loader

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.metrics import mean_squared_error as mse

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
years = '2018' # '2016' or '2018'
city = 'Seoul'
n_steps, length = 12, 12

In [ ]:
es = tf.keras.callbacks.EarlyStopping(patience=20, monitor='val_loss', restore_best_weights=True)
lr = tf.keras.callbacks.ReduceLROnPlateau(patience=5, monitor='val_loss', min_lr=1e-7)

In [ ]:
print(city, year)
datasets = data_loader('./datasets', city, year, length=length, n_steps=n_steps, is_scale=True)

# Extract Features to their corresponding variables
risk_train, risk_test = datasets['risk'][0], datasets['risk'][1]
demo_train, demo_test = datasets['demo'][0], datasets['demo'][1]
poi_train, poi_test = datasets['poi'][0], datasets['poi'][1]
road_train, road_test = datasets['road'][0], datasets['road'][1]
volume_train, volume_test = datasets['volume'][0], datasets['volume'][1]
weather_train, weather_test = datasets['weather'][0], datasets['weather'][1]
calendar_train, calendar_test = datasets['calendar'][0], datasets['calendar'][1]
c_train, c_test = datasets['dtg'][0], datasets['dtg'][1]
y_train, y_test = datasets['y'][0], datasets['y'][1]

# Join all static environmental features 
E_static_train = np.concatenate([demo_train, poi_train, road_train], axis=-1)
E_static_test = np.concatenate([demo_test, poi_test, road_test], axis=-1)

# Join all dynamic environmental features 
E_dynamic_train = np.concatenate([volume_train, weather_train, calendar_train], axis=-1)
E_dynamic_test = np.concatenate([volume_test, weather_test, calendar_test], axis=-1)

n_districts = len(datasets['selected_areas'])
n_features = {'risk': risk_train.shape[-1], 'E_static': E_static_train.shape[-1], 'E_dynamic': E_dynamic_train.shape[-1], 'c': c_train.shape[-1]}

X_train = [risk_train, E_static_train, E_dynamic_train, c_train]
X_test = [risk_test, E_static_test, E_dynamic_test, c_test]

model = DF_TAR(n_steps, length, n_districts, n_features)

model.compile(optimizer='adam', loss=['mae','mse'])
model.fit(X_train, [y_train, y_train], epochs=200, validation_split=0.05, callbacks=[es, lr], verbose=0)

y_pred = model.predict(X_test)
y_pred = final_score(y_pred, 0.25)

print(f'MAE: {mae(y_test.reshape(-1), y_pred.reshape(-1)):.4f}, RMSE: {mse(y_test.reshape(-1), y_pred.reshape(-1), squared=False):.4f}')